## 집 값 예측
- 예측할 변수 ['SalePrice']
- 평가: rmse, r2
    - rmse는 낮을 수록 좋은 성능
    - r2는 높을 수록 좋은 성능

In [ ]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='SalePrice', id_name='Id')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Data Load & Simple EDA

In [ ]:
import pandas as pd

In [ ]:
X_train.shape, X_test.shape

In [ ]:
pd.set_option("display.max_columns", 100)
display(X_train.head(3))
display(X_test.head(3))

In [ ]:
y_train['SalePrice'].hist()

In [ ]:
y_test['SalePrice'].hist()

In [ ]:
X_train.isnull().sum().sort_values(ascending=False)[:20]

In [ ]:
X_test.isnull().sum().sort_values(ascending=False)[:20]

In [ ]:
X_train.info()

# Preprocessing

In [ ]:
X_train = X_train.select_dtypes(exclude=['object'])
X_test = X_test.select_dtypes(exclude=['object'])
target = y_train['SalePrice']

In [ ]:
# id값 옮겨놓기
X_train_id = X_train.pop('Id')
X_test_id = X_test.pop('Id')

In [ ]:
X_train.head(1)

In [ ]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer()
X_train = imp.fit_transform(X_train)
X_test = imp.transform(X_test)

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, target, test_size=0.15, random_state=2022)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

# Model

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X_tr, y_tr, verbose=False)
pred = model.predict(X_val)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

print("RMSE : " + str(rmse(y_val, pred)))
print("R2 : " + str(r2_score(y_val, pred)))

# Simple Preprocessing

In [ ]:
X_train, X_test, y_train, y_test = exam_data_load(df, target='SalePrice', id_name='Id')

idx1 = y_train['SalePrice'].quantile(0.005)>y_train['SalePrice']
idx2 = y_train['SalePrice'].quantile(0.995)<y_train['SalePrice']

y_train = y_train[~(idx1 + idx2)]
X_train = X_train[~(idx1 + idx2)]

X_train = X_train.select_dtypes(exclude=['object'])
X_test = X_test.select_dtypes(exclude=['object'])
target = y_train['SalePrice']
X_train_id = X_train.pop('Id')
X_test_id = X_test.pop('Id')

imp = SimpleImputer()
X_train = imp.fit_transform(X_train)
X_test = imp.transform(X_test)

X_tr, X_val, y_tr, y_val = train_test_split(X_train, target, test_size=0.15, random_state=20222)

model = XGBRegressor()
model.fit(X_tr, y_tr)
pred = model.predict(X_val)

print("RMSE : " + str(rmse(y_val, pred)))
print("R2 : " + str(r2_score(y_val, pred)))

## Simple Tuning

In [ ]:
X_train, X_test, y_train, y_test = exam_data_load(df, target='SalePrice', id_name='Id')

idx1 = y_train['SalePrice'].quantile(0.005)>y_train['SalePrice']
idx2 = y_train['SalePrice'].quantile(0.995)<y_train['SalePrice']

y_train = y_train[~(idx1 + idx2)]
X_train = X_train[~(idx1 + idx2)]

X_train = X_train.select_dtypes(exclude=['object'])
X_test = X_test.select_dtypes(exclude=['object'])
target = y_train['SalePrice']
X_train_id = X_train.pop('Id')
X_test_id = X_test.pop('Id')

imp = SimpleImputer()
X_train = imp.fit_transform(X_train)
X_test = imp.transform(X_test)

X_tr, X_val, y_tr, y_val = train_test_split(X_train, target, test_size=0.15, random_state=20222)

model = XGBRegressor(n_estimators=100, max_depth=4, colsample_bytree=0.9)
model.fit(X_tr, y_tr)
pred = model.predict(X_val)

print("RMSE : " + str(rmse(y_val, pred)))
print("R2 : " + str(r2_score(y_val, pred)))

# Predict & to CSV

In [ ]:
pred = model.predict(X_test)
output = pd.DataFrame({'Id': X_test_id, 'SalePrice': pred})
output.head()

In [ ]:
# csv생성
output.to_csv("000000.csv", index=False)
output.head(3)

# 결과 체점

In [ ]:
pred = model.predict(X_test)
print("RMSE : " + str(rmse(y_test['SalePrice'], pred)))
print("R2 : " + str(r2_score(y_test['SalePrice'], pred)))